# To-do list

## Learning outcomes
To-Do list that will help you manage your tasks. You will practice using loops, conditions and statement branches. And also you will learn the basics of SQLAlchemy to manage SQLite database on python!

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Date
from datetime import datetime, timedelta

# create database
engine = create_engine('sqlite:///todo.db?check_same_thread=False')
Base = declarative_base()

# initiate the engine
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

class Table(Base):
    __tablename__ = 'task'
    id = Column(Integer, primary_key=True)
    task = Column(String)
    deadline = Column(Date, default=datetime.today().date())

    def __repr__(self):
        return self.task

    # task menu -- what do you want to do?
    def menu(self):
        while True:
            menu_items = int(input("1) Today's tasks\n2) Week's tasks\n3) All tasks\n4) Missed tasks\n5) Add task\n6) Delete task\n0) Exit\n"))
            if menu_items == 1:
                self.get_today()
            if menu_items == 2:
                self.get_week()
            if menu_items == 3:
                self.all_tasks()
            if menu_items == 4:
                self.missed_tasks()
            if menu_items == 5:
                self.add_task()
            if menu_items == 6:
                self.delete_task()
            elif menu_items == 0:
                print("Bye!")
                break

    # get all tasks that are due today
    def get_today(self):
        todays_date = datetime.today().date()
        all_tasks = session.query(Table).filter(Table.deadline == todays_date).all()
        if len(all_tasks) > 0:
            print("Today:\n")
            for i in range(len(all_tasks)):
                print(str(i+1)+".",all_tasks[i])
            print("")
        else:
            print("Nothing to do!")
        
    # get all tasks for the coming week
    def get_week(self):
        todays_date = datetime.today().date()
        week_later = todays_date + timedelta(days=6)
        delta = timedelta(days=1)
        while todays_date <= week_later:
            todays_task = session.query(Table).filter(Table.deadline == todays_date).all()
            print(todays_date.strftime("%A"),todays_date.day,todays_date.strftime('%b')+":")
            if len(todays_task) > 0:
                for i in range(len(todays_task)):
                    print(str(i+1)+".",todays_task[i])
            else:
                print("Nothing to do!")
            print("")
            todays_date += delta
        
    # print all tasks in table
    def all_tasks(self):
        all_tasks = session.query(Table).all()
        counter = 1
        print("All tasks:")
        if len(all_tasks) > 0:
            for task, deadline in session.query(Table.task, Table.deadline).order_by(Table.deadline):
                print(str(counter)+".",task+".",deadline.day,deadline.strftime('%b'))
                counter += 1
            print("")
        else:
            print("Nothing to do!")
         
    # did we miss any tasks?
    def missed_tasks(self):
        missed = session.query(Table).filter(Table.deadline < datetime.today().date()).all()
        counter = 1
        print("Missed tasks:")
        if len(missed) > 0:
            for task, deadline in session.query(Table.task, Table.deadline).order_by(Table.deadline).filter(Table.deadline < datetime.today().date()).all():
                print(str(counter)+".",task+".",deadline.day,deadline.strftime('%b'))
                counter += 1
            print("")
        else:
            print("Nothing is missed!")
            
    # add a task to the table
    def add_task(self):
        task_input = str(input("Enter task\n"))
        task_deadline = datetime.strptime(input("Enter deadline\n"), '%Y-%m-%d')
        new_task = Table(task=task_input, deadline = task_deadline) #datetime.today().date())
        session.add(new_task)
        print("The task has been added!\n")
        session.commit() 
     
    # delete a task from the table
    def delete_task(self):
        print("Chose the number of the task you want to delete:")
        get_tasks = session.query(Table).all()
        counter = 1
        if len(get_tasks) > 0:
            for task, deadline in session.query(Table.task, Table.deadline).order_by(Table.deadline):
                print(str(counter)+".",task+".",deadline.day,deadline.strftime('%b'))
                counter += 1
            remove_row = int(input())
            session.delete(get_tasks[remove_row])
            session.commit()
            print("The task has been deleted!")
        else:
            print("Nothing to do!")  

In [ ]:
# call the class
new_table = Table()
# pull up the task menu
new_table.menu()